# 🤖 RAG & Agent Cookbook: Automating Sales Follow-ups and Email Tasks



This notebook demonstrates an AI-powered assistant that automates **next-step extraction**, **email drafting**, and **email sending** from sales call transcripts using **OpenAI GPT-4o** and function calling.

---

🔍 **What this notebook does**:

- Takes a **sales conversation transcript** as input.
- Uses GPT-4o to **extract actionable next steps** (e.g., "Send proposal", "Follow up with finance").
- Automatically **drafts professional emails** for relevant tasks.
- Sends those emails via **SMTP** (e.g., Gmail).
- Uses modular **function calling tools** and **multi-step agent logic**.

---


## 🚀 Setup

### Install Dependencies
First, install the required packages and configure your environment.



In [ ]:
!pip install openai smtplib

## 🛠️ Install Required Libraries

In [4]:
import openai
import json
import smtplib
import os
from email.message import EmailMessage
import logging


## 🔐 Set Up Environment Variables

Configure your OpenAI API key and email credentials before running the notebook.

You can do this by creating a `.env` file or setting them directly in the notebook using `os.environ`.

### Required Variables:

- `OPENAI_API_KEY`: Your OpenAI API key.
- `EMAIL_USER`: Your email address (used to send emails).
- `EMAIL_PASS`: App password or SMTP password for your email.


In [18]:
# Set up API Key and Email Credentials
os.environ["OPENAI_API_KEY"] = "Your OpenAI API key."
os.environ["EMAIL_USER"] = "Your email address (used to send emails)."
os.environ["EMAIL_PASS"] = "App password or SMTP password for your email."


print("✅ Environment variables set.")

✅ Environment variables set.


## Loading Environment Variables


In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")
EMAIL_USER = os.getenv("EMAIL_USER")        # your email (e.g., your@email.com)
EMAIL_PASS = os.getenv("EMAIL_PASS")

# 🔑 Function Definitions
1. Extract Next Steps from a Transcript
This function uses GPT-4 to extract actionable next steps from a sales transcript.

In [9]:
def extractNextSteps(transcript: str) -> list:
    """
    Extract actionable next steps from a sales transcript.

    Args:
        transcript (str): The full transcript of the conversation.

    Returns:
        list: A list of next-step strings.
    """
    prompt = f"""
    You're a helpful sales assistant. Extract the next steps from this transcript as a bullet list.

    Transcript:
    {transcript}

    Only return the steps. No intro, no summary.
    """
    res = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    content = res.choices[0].message.content.strip()
    return [step.strip("-• ") for step in content.split("\n") if step.strip()]

2. Draft Email from Task
This function converts a task description into a professional email, inferring the recipient’s email if not explicitly mentioned.

In [10]:
def draftEmail(step: str) -> dict:
    """
    Draft a professional email based on a task/next step.

    Args:
        step (str): The next step/task to convert into an email.

    Returns:
        dict: A dictionary with 'to', 'subject', and 'body' keys.
    """

    prompt = f"""
    Convert this task into a professional email. If the recipient's email is not explicitly provided, infer it from the name or role mentioned. Default to "Your email" if uncertain.

    Task: {step}

    Format:
    To: [inferred or default email]
    Subject: ...
    Body:

    Best Regards
    Akshat Anand

    ...
    """

    res = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    lines = res.choices[0].message.content.splitlines()
    email = {"to": "", "subject": "", "body": ""}
    capture = None
    for line in lines:
        if line.lower().startswith("to:"):
            email["to"] = line[3:].strip()
            if not email["to"] or "@" not in email["to"]:
                email["to"] = "ayush@llumo.ai"
        elif line.lower().startswith("subject:"):
            email["subject"] = line[8:].strip()
            capture = "body"
        elif capture == "body":
            email["body"] += line.strip() + "\n"

    return email

3. Send the Email
The sendEmail function sends the email to the intended recipient.

In [11]:
def sendEmail(email: dict):
    """
    Send an email using SMTP.

    Args:
        email (dict): A dictionary with 'to', 'subject', and 'body' keys.

    Returns:
        dict: A dictionary with 'status' and 'error' keys.
    """
    try:
        msg = EmailMessage()
        msg["From"] = EMAIL_USER
        msg["To"] = email["to"]
        msg["Subject"] = email["subject"]
        msg.set_content(email["body"])

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(EMAIL_USER, EMAIL_PASS)
            smtp.send_message(msg)
        logging.info(f"✅ Email sent to {email['to']}")
        return {"status": "sent", "to": email["to"]}
    except Exception as e:
        logging.error(f"❌ Failed to send email: {e}")
        return {"status": "failed", "error": str(e)}


4. Tool Wrappers
These functions allow the tools to be executed and interact with the OpenAI API.

In [12]:
def tool_generateNextSteps(args):
    steps = extractNextSteps(args["transcript"])
    return json.dumps({"nextSteps": steps})

def tool_draftEmail(args):
    email = draftEmail(args["step"])
    return json.dumps(email)

def tool_sendEmail(args):
    result = sendEmail(args)
    return json.dumps(result)


5. Tool Definitions
Define the tools that will be used by the agent to carry out its tasks.

In [13]:
# === TOOL DEFINITIONS ===

tools = [
    {
        "type": "function",
        "function": {
            "name": "generateNextSteps",
            "description": "Extract next steps from a sales transcript.",
            "parameters": {
                "type": "object",
                "properties": {
                    "transcript": {"type": "string"}
                },
                "required": ["transcript"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "draftEmail",
            "description": "Turn a task into an email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "step": {"type": "string"}
                },
                "required": ["step"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "sendEmail",
            "description": "Send an email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string"},
                    "subject": {"type": "string"},
                    "body": {"type": "string"}
                },
                "required": ["to", "subject", "body"]
            }
        }
    }
]


6. Tool Call Executor
This part allows the agent to execute the tools by invoking them in the right order.

In [14]:
def executeTool(toolCall):
    """
    Executes a tool call using its function name and arguments.

    Args:
        toolCall: OpenAI tool call object.

    Returns:
        str: JSON result from tool execution.
    """
    args = json.loads(toolCall.function.arguments)
    if toolCall.function.name == "generateNextSteps":
        return tool_generateNextSteps(args)
    elif toolCall.function.name == "draftEmail":
        return tool_draftEmail(args)
    elif toolCall.function.name == "sendEmail":
        return tool_sendEmail(args)
    return json.dumps({"error": "Unknown tool"})


#### 🧪 Main Execution Loop
Run the agent that processes the transcript, extracts tasks, drafts emails, and sends them.

In [15]:
def runAgentWithTranscript(transcript: str):
    """
    Run the agent on a given transcript to extract next steps and send emails.

    Args:
        transcript (str): Sales transcript text.
    """
    messages = [{
        "role": "user",
        "content": f"Here is a transcript:\n{transcript}\n\nPlease extract next steps and send emails for any task that mentions emailing or sending."
    }]

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools
    )

    message = response.choices[0].message
    messages.append(message)

    # === Tool Call Execution Loop ===
    while message.tool_calls:
        for toolCall in message.tool_calls:
            result = executeTool(toolCall)
            logging.info(f"[TOOL: {toolCall.function.name}] result:\n{result}")
            messages.append({
                "role": "tool",
                "tool_call_id": toolCall.id,
                "content": result
            })

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools
        )
        message = response.choices[0].message
        messages.append(message)

    print("\n[FINAL OUTPUT]:", message.content or "[No content returned]")


#### 🔁 Example Usage
Now let’s run the agent with an example transcript.

In [17]:
if __name__ == "__main__":

    transcript = """
Harsh: Hey Ayush, good discussion today. Can you send the revised proposal to me at test@gmail.com by Friday evening? I need to go through it before our Monday call.

Ayush: Sure, I’ll send it across by EOD Friday.

Harsh: Also, can you follow up with finance at test@gmailcom about the discount approval? They haven't responded yet.

Ayush: Yeah, I’ll send them a nudge.

Harsh: Perfect. Lastly, let’s aim for a product demo with the client team next Tuesday at 11 AM. I’ll check if they’re available.

Ayush: Alright, I’ll tentatively block the slot.
"""

    runAgentWithTranscript(transcript)


[FINAL OUTPUT]: The tasks have been completed and the following emails have been sent:

1. **To Harsh at test@gmail.com:**  
   - **Subject:** Submission of Revised Proposal
   - **Email Content:**  
     Hi Harsh,
     
     I hope this message finds you well. I wanted to inform you that the revised proposal has been completed. I will be sending it over to you by Friday evening, as discussed.
     
     Please let me know if there are any other requirements or further adjustments needed.
     
     Thank you for your patience and cooperation.
     
     Best Regards,  
     Akshat Anand

2. **To Finance Team at test@gmailcom:**  
   - **Subject:** Inquiry on Discount Approval
   - **Email Content:**  
     Dear Finance Team,
     
     I hope this message finds you well. I am writing to follow up on the status of the discount approval process. Could you please provide an update at your earliest convenience? Your assistance in this matter is greatly appreciated.
     
     Thank you f